# Extract the dataset and load it to the database
For each .csv file the following apply: <br>
- each row represents the data of a single vehicle
- the first 10 columns in the 1st row include the columns’ names
- the first 4 columns include information about the trajectory like the unique trackID, the type of vehicle, <br>the distance traveled in meters and the average speed of the vehicle in km/h
- the last 6 columns are then repeated every 6 columns based on the time frequency. For example, column_5 <br> contains the latitude of the vehicle at time column_10, and column­­­_11 contains the latitude of the vehicle at time column_16.
- Speed is in km/h, Longitudinal and Lateral Acceleration in m/sec2 and time in seconds.

### Import packages

In [3]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [5]:
from utils import run_sql_query, populate_dataframe_to_database

### Extract th csv file

In [7]:
with open("../data/dataset.csv", 'r') as file:
    lines = file.readlines()

In [10]:
lines[1]

'1; Car; 48.85; 9.770344; 37.977391; 23.737688; 4.9178; 0.0518; -0.0299; 0.000000; 37.977391; 23.737688; 4.9207; -0.0124; -0.0354; 0.040000; 37.977391; 23.737688; 4.9160; -0.0519; -0.0413; 0.080000; 37.977390; 23.737688; 4.9057; -0.0914; -0.0478; 0.120000; 37.977390; 23.737689; 4.8871; -0.1679; -0.0550; 0.160000; 37.977389; 23.737689; 4.8547; -0.2815; -0.0628; 0.200000; 37.977389; 23.737689; 4.8033; -0.4320; -0.0713; 0.240000; 37.977388; 23.737690; 4.7318; -0.5617; -0.0804; 0.280000; 37.977388; 23.737690; 4.6442; -0.6543; -0.0905; 0.320000; 37.977388; 23.737690; 4.5437; -0.7413; -0.1017; 0.360000; 37.977387; 23.737690; 4.4311; -0.8227; -0.1143; 0.400000; 37.977387; 23.737691; 4.3046; -0.9354; -0.1285; 0.440000; 37.977386; 23.737691; 4.1600; -1.0729; -0.1439; 0.480000; 37.977386; 23.737691; 4.4411; -0.5617; -0.0949; 0.520000; 37.977385; 23.737692; 4.3561; -0.6183; -0.1044; 0.560000; 37.977385; 23.737692; 4.2639; -0.6621; -0.1133; 0.600000; 37.977384; 23.737692; 4.1654; -0.7059; -0.1219;

In [11]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
cols = lines_as_lists.pop(0)

In [12]:
vehicle_col = cols[:4]
trejictory_col = ['track_id'] + cols[4:]

print(vehicle_col)
print(trejictory_col)

['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [13]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4])

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix


In [15]:
#now lets convert them into dataframes
vehicle_data = pd.DataFrame(track_info, columns=vehicle_col)
trajectory_data = pd.DataFrame(trajectory_info, columns=trejictory_col)

In [16]:
vehicle_data.head()

,track_id,type,traveled_d,avg_speed
0,1,Car,48.85,9.770344
1,2,Motorcycle,98.09,19.839417
2,3,Motorcycle,63.80,18.228752
3,4,Motorcycle,145.72,26.229014
4,5,Motorcycle,138.01,24.841425


In [17]:
vehicle_data.shape

(922, 4)

In [18]:
trajectory_data.head()

,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.977391,23.737688,4.9178,0.0518,-0.0299,0.000000
1,1,37.977391,23.737688,4.9207,-0.0124,-0.0354,0.040000
2,1,37.977391,23.737688,4.9160,-0.0519,-0.0413,0.080000
3,1,37.977390,23.737688,4.9057,-0.0914,-0.0478,0.120000
4,1,37.977390,23.737689,4.8871,-0.1679,-0.0550,0.160000


In [19]:
trajectory_data.shape

(1446887, 7)

In [20]:
vehicle_data.to_csv("../data/vehicle_data.csv")
trajectory_data.to_csv("../data/trajectory_data.csv")

### Load the dataset into the database

In [24]:
connection_params = {
    "host": "localhost",
    "user": "airflow",
    "password": "airflow",
    "port": "5432",
    "database": "airflow"  # Change to your desired database name
}

vehicle_data_table_name = "vehicle_data"
vehicle_trajectory_table_name = "vehicle_trajectory"

create_vehicle_table_query =  '''
    CREATE TABLE IF NOT EXISTS vehicle_data (
        "track_id" bigint PRIMARY KEY,
        "type" text,
        "traveled_d" double precision,
        "avg_speed" double precision
    );
'''
create_trajectory_table_query =  '''
    CREATE TABLE IF NOT EXISTS vehicle_trajectory (
        "detail_id" serial PRIMARY KEY,
        "track_id" bigint REFERENCES vehicle_data("track_id"),
        "lat" double precision,
        "lon" double precision,
        "speed" double precision,
        "lon_acc" double precision,
        "lat_acc" double precision,
        "time" double precision
);
'''




In [22]:
run_sql_query(connection_params, create_vehicle_table_query)
run_sql_query(connection_params, create_trajectory_table_query)


Log success
Log success


In [23]:
populate_dataframe_to_database(connection_params, vehicle_data, vehicle_data_table_name)

Inserted 922 rows into the database table vehicle_data.


In [25]:
populate_dataframe_to_database(connection_params, trajectory_data, vehicle_trajectory_table_name)

Inserted 1446887 rows into the database table vehicle_trajectory.
